# Decision making under uncertainty: optimising reservoir pumped inflow scheduling under uncertain hydrological forecasts
In this Notebook we will see how to deal with uncertainty in simulation and optimisation models when supporting operational decisions for a water reservoir system.

<left><img src="../../util/images/Dam2.gif" width = "600px"><left>
    
We consider again a simple illustrative system of a reservoir operated to supply water to a domestic consumption node. This time however we assume that the reservoir inflows can be augmented by pumping river abstractions into the reservoir.

<left> <img src="../../util/images/system_representation_IO3.png" width = "600px"><left>

Again, we use a mathematical model to simulate the system and find optimal operations. This time, the decision to be made is about how much water to pump into the reservoir (***Qreg_inf***). So, the objective is to determine the pumping scheduling that will minimise pumping costs, while meeting the water demand (***d***) over a coming period of time (we will assume here that the regulated release ***Qreg_rel*** is always set to the demand ***d***, unless physically impossible, that is, unless the reservoir storage is too low and the release must be reduced; in other words, we assume there is no **hedging** in this case). 

<left> <img src="../../util/images/uncertainty.1.jpg" width = "400px"><left>
    
We will run optimisation against some forecasted inflows and demands. In this Notebook, we will use uncertain forecasts - specifically: an ensemble of several possible scenarios with the same probability of occurrence ([Learn more about ensemble prediction](https://www.youtube.com/watch?v=NLhRUun2iso)) - and we will investigate how to deal with such uncertainties when making decisions. We will look at two possible approaches: one ('deterministic') where uncertainty is reduced a priori (=before running optimisation) from the user; and one ('stochastic') where instead uncertainty is explicitely considered within the optimisation process. We will also evaluate each of these approaches by looking at how the decisions made perform once implemented against the actual inflows and demands, instead of forecasts.

But first of all, we need to import the necessary libraries to run the model simulation and optimisation:

In [1]:
from bqplot import pyplot as plt
from bqplot import *
from bqplot.traits import *
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from platypus import NSGAII, Problem, Real, Integer

from Modules.Forecast_ensemble import Ensemble_member_sel, Observed_inflows, Forecast_ensemble
from Modules.Interactive_pump_schedule import Interactive_Pareto_front_det, Interactive_Pareto_front_act, Interactive_Pareto_front
from Modules.clim_dem_forecast import forecast
warnings.filterwarnings('ignore') # to ignore warning messages

## Deterministic approach: pre-filtering uncertainty

#### Definition of inflow and demand scenarios
Let's consider a forecast period of 8 weeks, and load the uncertain forecasts of inflows (I) and water demand (d) for this period. Uncertainty in the forecasts is represented by an ensemble of 10 possible time series.

In [2]:
N = 8 # (weeks) length of the simulation period
M = 10 # number of members in the ensemble 
I_fore,d_fore = forecast(N,M) # (ML/day) generate forecast ensemble
e_fore = 3 + np.zeros([M,N]) # We assume the evaporation as constant and equal to 3 ML/week

Plot the inflow and demand forecasts: Among the 10 possible time series, i.e. ensemble members, choose one inflow time series and one demand time series according to your criteria. NOTE: make sure that you have selected one of the forecast members so you can see the number of the selected member on the figure's title.

In [3]:
fig_1a,fig_1b,I_sel,d_sel = Ensemble_member_sel(N,M,I_fore,d_fore)
widgets.VBox([fig_1a,fig_1b])

**Note** - make sure the line has been actually selected: when it has, you will see the number of the chosen forecast member displayed in the Figure title.

#### Definition of other input parameters 
Let's define other variables that are needed for the reservoir system simulation, such as the reservoir storage capacity, the environmental compensation flow, etc.

In [4]:
### Constraints ###
Smax = 150 #  (ML) Maximum storage (=reservoir capacity)
Smin = 0 # (ML) Minimum storage (set to zero for now)
env_min = 2 # (ML/week)   # Environmental compensation flow

### Initial conditions ###
S0 = 40 # (ML) Storage volume at the beginning of the simulation period

### Other parameters ### 
c = 50 # (£/ML) Pumping energy cost

#### Implementation of the reservoir simulation function
Here we define a function that implements the reservoir simulation, that is, iteratively apply the mass balance equation and reconstruct the temporal evolution of the reservoir variables over the simulation period

In [5]:
from Modules.Water_system_model import Water_system_model as syst_sim

#### Two-objective optimization

The following code will run an automatic optimisation of the reservoir system, considering two objectives: Total Squared Deficit (TSD) and Total Pumping Costs (TPC):

$$TSD = \sum_{t=1}^{N} [ \ max( \ 0, \ d(t)-Qreg\_rel(t) \ ) \ ]^2 $$
$$TPC = \sum_{t=1}^{N} [Qreg\_inf(t)*c] $$

The optimisation will use one forcing time series for the inflows (I) and the demand (d), out of the 10 available in the ensemble. You can choose which time series will be used by clicking with the mouse on one of the lines in the Inflow forecast and Demand forecast figures above.

In [6]:
# Optimizer
def auto_optim(vars):
    
    pinflow1 = vars[0]
    pinflow2 = vars[1]
    pinflow3 = vars[2]
    pinflow4 = vars[3]
    pinflow5 = 0
    pinflow6 = 0
    pinflow7 = 0
    pinflow8 = 0
    
    Qreg_inf = np.array([pinflow1,pinflow2,pinflow3,pinflow4,pinflow5,pinflow6,pinflow7,pinflow8])
    S,env,spill,Qreg_rel = syst_sim(N,I_sel+Qreg_inf,e_fore,d_sel,S0,Smax,env_min)
    
    sdpen = (np.sum((np.maximum(d_sel-Qreg_rel,[0]*N))**2)).astype('int')
    pcost = (np.sum(np.array(Qreg_inf)*c)).astype('int')
    
    return [sdpen,pcost]

problem = Problem(4,2)
Real0 = Real(0, 40); Real1 = Real(0, 40); Real2 = Real(0, 40); Real3 = Real(0, 40)

problem.types[:] = [Real0] + [Real1] + [Real2] + [Real3]
problem.function = auto_optim

population_size = 20 # Number of candidate solutions evaluated at each iteration
algorithm = NSGAII(problem,population_size)
algorithm.run(10000) # Number of iterations

results1_optim_relea_2 = np.array([algorithm.result[i].objectives[0] for i in range(population_size)])
results2_optim_relea_2 = np.array([algorithm.result[i].objectives[1] for i in range(population_size)])

solutions_optim_relea_2 = [[algorithm.result[i].variables[0],algorithm.result[i].variables[1],algorithm.result[i].variables[2],
                            algorithm.result[i].variables[3],0,0,0,0] for i in range(population_size)]

#### Plot the optimisation results
We can visualise the tradeoffs between the two objectives in the Pareto front plot, which displays the combination of the two objective values in correspondence to a set of optimised solutions. Click on one point in the Pareto front to visualise the pumping scheduling that generates that performance, the associated time series of reservoir storages and releases, and some more information about the total pumped inflow and deficit volume.

In [7]:
fig_2pf,fig_2b,fig_2c,fig_2d,pareto_det = Interactive_Pareto_front_det(N,I_sel,e_fore,d_sel,S0,Smax,Smin,env_min,c, 
                        solutions_optim_relea_2,results1_optim_relea_2,results2_optim_relea_2)
pareto_det.colors=['deepskyblue']
widgets.VBox([widgets.HBox([widgets.VBox([fig_2d,fig_2b]),fig_2pf]),widgets.HBox([fig_2c])])

**Let's think about this question**:

Depending on the inflow and demand forecast members selected previously, do the Pareto front axis change? why is that?


## Deterministic approach: evaluation
After 8 weeks, we can evaluate the quality of our optimised decisions against the inflows and demands that actually occured.

<img src="../../util/images/Calendar2.jpg" width="400px"/>


First, we load and plot the inflow and demand observations for our 8 weeks period (for convenience, the plot also report the ensemble forecast that we were presented previously and, highlighted in bold, the ensemble member that we selected for the deterministic optimisation):

In [8]:
I_act,T_act,e_act,d_act,fig_3a,fig_3b = Observed_inflows(N,M,I_sel,d_sel,I_fore,d_fore)
widgets.VBox([fig_3a,fig_3b])

Now we can simulate the system performance when implementing the pumping scheduling against the observed inflows and demands. For convenience, the figure below shows the Pareto front that we obtained from optimisation, including the point (in red) that we had chosen at the end of the previous step; and the ***actual*** performance delivered by that solution once evaluated against the observed inflows and demands (in black). The other plot shows the actual time series of reservoir releases and storages.

In [9]:
fig_4b,fig_4c,fig_4d,fig_4pf = Interactive_Pareto_front_act(N,I_act,e_act,d_act,S0,Smax,Smin,env_min,c,
                                            solutions_optim_relea_2,results1_optim_relea_2,results2_optim_relea_2,pareto_det.selected[0])
widgets.VBox([widgets.HBox([widgets.VBox([fig_4d,fig_4b]),fig_4pf]),widgets.HBox([fig_4c])])

**Let's think about these questions:**: <br>
* Did you achieve more or less than what forecasted? why is that?<br>
* Going back to the **Definition of inflow and demand scenarios** stage, would you choose a different ensemble member to run the optimisation? (You can actually do this by changing your choices above and re-running the code!) <br>
* Going back to the Optimisation stage, would you choose a different Pareto front point? (You can actually do this by changing your choices above and re-running the code!) <br>
* What happen when you assume a more pessimistic or more optimistic scenario? <br>
* Does uncertainty have the same impacts on all tradeoff solutions in the Pareto front?

## Stochastic approach: including uncertainty in the optimisation
In this approach, instead of pre-filtering the uncertainty by selecting one ensemble member to optimise against, we carry over the entire ensemble into the optimisation procedure. The optimisation algorithm will therefore look for solutions that perform 'best' (on average) against all the ensemble members, instead of doing best for one specific ensemble member:

In [10]:
from platypus import NSGAII, Problem, Real, Integer

def auto_optim_v2(vars):
    
    pinflow1 = vars[0]
    pinflow2 = vars[1]
    pinflow3 = vars[2]
    pinflow4 = vars[3]
    pinflow5 = 0
    pinflow6 = 0
    pinflow7 = 0
    pinflow8 = 0
    
    Qreg_inf = np.array([pinflow1,pinflow2,pinflow3,pinflow4,pinflow5,pinflow6,pinflow7,pinflow8])
    S,env,spill,Qreg_rel = syst_sim(N,I_fore+Qreg_inf,e_fore,d_fore,S0,Smax,env_min)
    
    sdpen_mean = np.mean(np.sum(np.maximum(d_fore-Qreg_rel,np.zeros(np.shape(d_fore)))**2,axis=1))
    pcost = np.sum(np.array(Qreg_inf)*c)
    
    return [sdpen_mean,pcost]

problem = Problem(4,2)
Real0 = Real(0, 40); Real1 = Real(0, 40); Real2 = Real(0, 40); Real3 = Real(0, 40)

problem.types[:] = [Real0] + [Real1] + [Real2] + [Real3]
problem.function = auto_optim_v2

population_size = 20 # Number of candidate solutions evaluated at each iteration
algorithm = NSGAII(problem,population_size)
algorithm.run(10000) # Number of iterations

results1_optim_relea = np.array([algorithm.result[i].objectives[0] for i in range(population_size)])
results2_optim_relea = np.array([algorithm.result[i].objectives[1] for i in range(population_size)])

solutions_optim_relea = [[algorithm.result[i].variables[0],algorithm.result[i].variables[1],
                             algorithm.result[i].variables[2],algorithm.result[i].variables[3],
                             0,0,0,0] for i in range(population_size)]

#### Plot the optimisation results
**Comment:** To represent the uncertainty of the forecasts the color intensity of the shaded areas varies according to the number of members of the forecast ensemble in which a certain value is reached.

In [11]:
fig_pf,fig_wd,fig_st,fig_in,pareto_ens = Interactive_Pareto_front(
    N,I_fore,e_fore,d_fore,S0,Smax,Smin,env_min,c,solutions_optim_relea,results1_optim_relea,results2_optim_relea)
widgets.VBox([widgets.HBox([widgets.VBox([fig_in,fig_wd]),fig_pf]),widgets.HBox([fig_st])])

***Comments:*** Let's think about these questions 

* Why there are more red points in the Pareto front of the stochastic approach compared to the deterministic approach?


## Stochastic approach: evaluation
After 8 weeks, we can evaluate the quality of our optimised decisions against the inflows and demands that actually occured. For convenience, the figure below shows the Pareto front that we obtained from optimisation, including the point (in red) that we had chosen; and the ***actual*** performance delivered by that solution once evaluated against the observed inflows and demands (in black). The other plot shows the actual time series of reservoir releases, supply deficits and storages.

<img src="../../util/images/Calendar2.jpg" width="400px"/>

In [12]:
fig_wd_act,fig_st_act,fig_in_act,fig_pf_act = Interactive_Pareto_front_act(N,I_act,e_act,d_act,S0,Smax,Smin,env_min,c,
                                            solutions_optim_relea,results1_optim_relea,results2_optim_relea,pareto_ens.selected[0])
widgets.VBox([widgets.HBox([widgets.VBox([fig_in_act,fig_wd_act]),fig_pf_act]),widgets.HBox([fig_st_act])])

***Let's think about these questions***: <br>
* Did you achieve more or less than what forecasted? why is that?<br>
* Going back to the Optimisation stage, would you choose a different Pareto front point? (You can actually do this by changing your choices above and re-running the code!) <br>
* What happen when you assume a more conservative or more risky decision? <br>
* Does uncertainty have the same impacts on all tradeoff solutions in the Pareto front? <br>
* Does the stochastic approach substracts part of the subjective component of the process? if so, which one?
* Under what conditions the deterministic approach is better than the stochastic and viceversa?